# Long reads to correct assembly

* Long read correction is recommended to correct the assembly before short read correction
* Program: RACON
* Input file is Shasta and Flye contig sets
* Ann did four rounds of RACON corrections, how many we need? Do we need other correctors as well?

## 1. input file paths

In [36]:
ONT=/output/genomic/plant/Vaccinium/myrtillus/Genome/ONT/CleanReads/All_AdapterRemoved.fastq
SHASTA_CTG=/workspace/hraczw/github/GA/Bilberry_genome/100.all.assemblies/Shasta_all_v0.2.0.assembly.ml1000.fasta.fasta
FLYE_CTG=/workspace/hraczw/github/GA/Bilberry_genome/100.all.assemblies/Flye_all_trimmed_i1.assembly.ml1000.fasta.fasta

In [3]:
mkdir 150.RACON.correction

In [37]:
WORKDIR=/workspace/hraczw/github/GA/Bilberry_genome/150.RACON.correction

## 2. aligning long reads to contigs

### 2.1 BWA

In [2]:
module load bwa
module list

Currently Loaded Modulefiles:
  1) powerPlant/core    4) git/2.21.0         7) asub/2.1
  2) texlive/20151117   5) perlbrew/0.76      8) bwa/0.7.17
  3) pandoc/1.19.2      6) perl/5.28.0


In [7]:
bwa index 


Usage:   bwa index [options] <in.fasta>

Options: -a STR    BWT construction algorithm: bwtsw, is or rb2 [auto]
         -p STR    prefix of the index [same as fasta name]
         -b INT    block size for the bwtsw algorithm (effective with -a bwtsw) [10000000]
         -6        index files named as <in.fasta>.64.* instead of <in.fasta>.* 

         `-a div' do not work not for long genomes.



: 1

In [8]:
bsub -J index \
-o $WORKDIR/index_shasta.out \
-e $WORKDIR/index_shasta.err \
"bwa index $SHASTA_CTG"

Job <739023> is submitted to default queue <lowpriority>.


In [9]:
bsub -J index \
-o $WORKDIR/index_flye.out \
-e $WORKDIR/index_flye.err \
"bwa index $FLYE_CTG"

Job <739024> is submitted to default queue <lowpriority>.


In [10]:
bsub -J mapping \
-n 20 \
-o $WORKDIR/mapping_shasta.out \
-e $WORKDIR/mapping_shasta.err \
"bwa mem \
-t 20 \
-x ont2d \
$SHASTA_CTG \
$ONT >\
$WORKDIR/ONT_to_shasta.sam"

Job <739135> is submitted to default queue <lowpriority>.


In [11]:
bsub -J mapping \
-n 20 \
-o $WORKDIR/mapping_flye.out \
-e $WORKDIR/mapping_flye.err \
"bwa mem \
-t 20 \
-x ont2d \
$FLYE_CTG \
$ONT >\
$WORKDIR/ONT_to_flye.sam"

Job <739136> is submitted to default queue <lowpriority>.


NOTE: mapping is very slow and sam output doesn't work with Racon. There are no empty lines in the assembly or the read files, not sure why.

Error message:
[racon::Polisher::initialize] loaded target sequences 7.202108 s
[racon::Polisher::initialize] loaded sequences 1127.074367 s
terminate called after throwing an instance of 'std::invalid_argument'
  what():  [bioparser::SamParser] error: invalid file format!
/home/hraczw/.lsbatch/1590894810.744722: line 8: 41504 Aborted                 /workspace/hraegb/singularity/racon-1.4.7.sif /output/genomic/plant/Vaccinium/myrtillus/Genome/ONT/CleanReads/All_AdapterRemoved.fastq.gz /workspace/hraczw/github/GA/Bilberry_genome/150.RACON.correction/ONT_to_flye.sam /workspace/hraczw/github/GA/Bilberry_genome/100.all.assemblies/Flye_all_trimmed_i1.assembly.fasta

TRY other mappers!

### 2.2 minimap2

In [23]:
module load minimap2
module list

Currently Loaded Modulefiles:
  1) powerPlant/core    4) git/2.21.0         7) asub/2.1
  2) texlive/20151117   5) perlbrew/0.76      8) minimap2/2.9
  3) pandoc/1.19.2      6) perl/5.28.0        9) singularity/3


In [3]:
minimap2 -h

Usage: minimap2 [options] <target.fa>|<target.idx> [query.fa] [...]
Options:
  Indexing:
    -H           use homopolymer-compressed k-mer
    -k INT       k-mer size (no larger than 28) [15]
    -w INT       minizer window size [10]
    -I NUM       split index for every ~NUM input bases [4G]
    -d FILE      dump index to FILE []
  Mapping:
    -f FLOAT     filter out top FLOAT fraction of repetitive minimizers [0.0002]
    -g NUM       stop chain enlongation if there are no minimizers in INT-bp [5000]
    -G NUM       max intron length (effective with -xsplice; changing -r) [200k]
    -F NUM       max fragment length (effective with -xsr or in the fragment mode) [800]
    -r NUM       bandwidth used in chaining and DP-based alignment [500]
    -n INT       minimal number of minimizers on a chain [3]
    -m INT       minimal chaining score (matching bases minus log gap penalty) [40]
    -X           skip self and dual mappings (for the all-vs-all mode)
    -p FLOAT     min secondary-

In [6]:
bsub -J minimap2 \
-n 20 \
-o $WORKDIR/mapping_minimap2_flye.out \
-e $WORKDIR/mapping_minimap2_flye.err \
"minimap2 \
-x map-ont \
-t 20 \
$FLYE_CTG \
$ONT \
> $WORKDIR/ONT_to_flye.minimap2.paf"

Job <746066> is submitted to default queue <lowpriority>.


In [7]:
bsub -J minimap2 \
-n 20 \
-o $WORKDIR/mapping_minimap2_shasta.out \
-e $WORKDIR/mapping_minimap2_shasta.err \
"minimap2 \
-x map-ont \
-t 20 \
$SHASTA_CTG \
$ONT \
> $WORKDIR/ONT_to_shasta.minimap2.paf"

Job <746067> is submitted to default queue <lowpriority>.


NOTE: mapping is way faster than BWA mem showing above and Racon-gpu works with .paf file

In [12]:
rm $WORKDIR/ONT_to_flye.sam $WORKDIR/ONT_to_shasta.sam

## 3. Racon-gpu

In [8]:
module load singularity
export SINGULARITY_NV=true

In [9]:
/workspace/hraegb/singularity/racon-1.4.7.sif -h

usage: racon [options ...] <sequences> <overlaps> <target sequences>

    <sequences>
        input file in FASTA/FASTQ format (can be compressed with gzip)
        containing sequences used for correction
    <overlaps>
        input file in MHAP/PAF/SAM format (can be compressed with gzip)
        containing overlaps between sequences and target sequences
    <target sequences>
        input file in FASTA/FASTQ format (can be compressed with gzip)
        containing sequences which will be corrected

    options:
        -u, --include-unpolished
            output unpolished target sequences
        -f, --fragment-correction
            perform fragment correction instead of contig polishing
            (overlaps file should contain dual/self overlaps!)
        -w, --window-length <int>
            default: 500
            size of window on which POA is performed
        -q, --quality-threshold <float>
            default: 10.0
            threshold for average base quality of window

**NOTE:** in order to run racon on GPU you need to specify the number of batches for the POA part and optionally the number of batches for the alignment part. You can also use banded approach on the GPU if you want to. Bellow are the options you need to set. I have updated the README as some of them were incomplete. POA batches take around 2.2GB of memory so you have to set that accordingly with your GPU. For the alignment batches I am not sure. Maybe @tijyojwad and @vellamike can help out.

-c, --cudapoa-batches <int>
    default: 0
    number of batches for CUDA accelerated polishing
-b, --cuda-banded-alignment
    use banding approximation for polishing on GPU. Only applicable when -c is used.
--cudaaligner-batches <int>
    default: 0
    number of batches for CUDA accelerated alignment
    
Setting -c and --cudaaligner-batches to non-zero will enable GPU acceleration for those steps accordingly.

For accelerated POA racon uses 90% of available memory and then distributes that within the specified batches.
For accelerated alignment each batch takes up a fixed amount of memory, so memory scales up as more batches are added.
    
The batch size for alignment on the GPU is currently hard coded, so each batch will take around 700MB irrespective of the properties of the data.
The CUDA accelerated alignment I'd say is still in beta phase, and undergoing some significant improvements. We'll update the racon integration to be more aware of the coverage levels and average read lengths, but for now it's static.
The batch size can be updated in code (https://github.com/lbcb-sci/racon/blob/master/src/cuda/cudapolisher.cpp#L91) and the upper limit on read length per batch is specified at https://github.com/lbcb-sci/racon/blob/master/src/cuda/cudapolisher.cpp#L169

### 3.1 round 1

In [21]:
bsub -J racon-gpu \
-m wkoppg31 \
-o $WORKDIR/racon_i1_flye_gpu.out \
-e $WORKDIR/racon_i1_flye_gpu.err \
-R gpu \
"/workspace/hraegb/singularity/racon-1.4.7.sif \
--include-unpolished \
-c 2 \
-b \
--cudaaligner-batches 2 \
$ONT \
$WORKDIR/ONT_to_flye.minimap2.paf \
$FLYE_CTG > $WORKDIR/Flye_all_trimmed_i1.assembly.racon_i1.includeUnpolished.gpu.fasta"

Job <746094> is submitted to default queue <lowpriority>.


45250.67 sec

In [14]:
bsub -J racon-cpu \
-n 40 \
-o $WORKDIR/racon_i1_flye_cpu.out \
-e $WORKDIR/racon_i1_flye_cpu.err \
-R gpu \
"/workspace/hraegb/singularity/racon-1.4.7.sif \
--include-unpolished \
-t 40 \
$ONT \
$WORKDIR/ONT_to_flye.minimap2.paf \
$FLYE_CTG > $WORKDIR/Flye_all_trimmed_i1.assembly.racon_i1.includeUnpolished.cpu.fasta"

Job <746078> is submitted to default queue <lowpriority>.


175840.97 sec

In [22]:
bsub -J racon-gpu \
-o $WORKDIR/racon_i1_shasta_gpu.out \
-e $WORKDIR/racon_i1_shasta_gpu.err \
-m wkoppg34 \
-R gpu \
"/workspace/hraegb/singularity/racon-1.4.7.sif \
--include-unpolished \
-c 2 \
-b \
--cudaaligner-batches 2 \
$ONT \
$WORKDIR/ONT_to_shasta.minimap2.paf \
$SHASTA_CTG > $WORKDIR/Shasta_all_v0.2.0.assembly.racon_i1.includeUnpolished.gpu.fasta"

Job <746095> is submitted to default queue <lowpriority>.


42705.11

In [24]:
FLYE_CTG_RACON_I1_GPU=$WORKDIR/Flye_all_trimmed_i1.assembly.racon_i1.includeUnpolished.gpu.fasta
FLYE_CTG_RACON_I1_CPU=$WORKDIR/Flye_all_trimmed_i1.assembly.racon_i1.includeUnpolished.cpu.fasta
SHASTA_CTG_RACON_I1_GPU=$WORKDIR/Shasta_all_v0.2.0.assembly.racon_i1.includeUnpolished.gpu.fasta

### 3.2 round 2

#### 3.2.1 mapping

In [25]:
bsub -J minimap2 \
-n 20 \
-o $WORKDIR/mapping_minimap2_flye_i1_gpu.out \
-e $WORKDIR/mapping_minimap2_flye_i1_gpu.err \
"minimap2 \
-x map-ont \
-t 20 \
$FLYE_CTG_RACON_I1_GPU \
$ONT \
> $WORKDIR/ONT_to_flye_i1_gpu.minimap2.paf"

Job <746100> is submitted to default queue <lowpriority>.


In [26]:
bsub -J minimap2 \
-n 20 \
-o $WORKDIR/mapping_minimap2_flye_i1_cpu.out \
-e $WORKDIR/mapping_minimap2_flye_i1_cpu.err \
"minimap2 \
-x map-ont \
-t 20 \
$FLYE_CTG_RACON_I1_CPU \
$ONT \
> $WORKDIR/ONT_to_flye_i1_cpu.minimap2.paf"

Job <746101> is submitted to default queue <lowpriority>.


In [27]:
bsub -J minimap2 \
-n 20 \
-o $WORKDIR/mapping_minimap2_shasta_i1_gpu.out \
-e $WORKDIR/mapping_minimap2_shasta_i1_gpu.err \
"minimap2 \
-x map-ont \
-t 20 \
$SHASTA_CTG_RACON_I1_GPU \
$ONT \
> $WORKDIR/ONT_to_shasta_i1_gpu.minimap2.paf"

Job <746102> is submitted to default queue <lowpriority>.


#### 3.2.2 racon

In [28]:
bsub -J racon-gpu \
-m wkoppg31 \
-o $WORKDIR/racon_i2_flye_gpu.out \
-e $WORKDIR/racon_i2_flye_gpu.err \
-R gpu \
"/workspace/hraegb/singularity/racon-1.4.7.sif \
--include-unpolished \
-c 2 \
-b \
--cudaaligner-batches 2 \
$ONT \
$WORKDIR/ONT_to_flye_i1_gpu.minimap2.paf \
$FLYE_CTG_RACON_I1_GPU > $WORKDIR/Flye_all_trimmed_i1.assembly.racon_i2.includeUnpolished.gpu.fasta"

Job <746106> is submitted to default queue <lowpriority>.


In [31]:
$WORKDIR

bash: /workspace/hraczw/github/GA/Bilberry_genome/150.RACON.correction: Is a directory


: 126

43856.73 sec

In [29]:
bsub -J racon-cpu \
-n 40 \
-m wkoppg32 \
-o $WORKDIR/racon_i2_flye_cpu.out \
-e $WORKDIR/racon_i2_flye_cpu.err \
-R gpu \
"/workspace/hraegb/singularity/racon-1.4.7.sif \
--include-unpolished \
-t 40 \
$ONT \
$WORKDIR/ONT_to_flye_i1_cpu.minimap2.paf \
$FLYE_CTG_RACON_I1_CPU > $WORKDIR/Flye_all_trimmed_i1.assembly.racon_i2.includeUnpolished.cpu.fasta"

Job <746107> is submitted to default queue <lowpriority>.


176357.41 sec

In [30]:
bsub -J racon-gpu \
-o $WORKDIR/racon_i2_shasta_gpu.out \
-e $WORKDIR/racon_i2_shasta_gpu.err \
-m wkoppg34 \
-R gpu \
"/workspace/hraegb/singularity/racon-1.4.7.sif \
--include-unpolished \
-c 2 \
-b \
--cudaaligner-batches 2 \
$ONT \
$WORKDIR/ONT_to_shasta_i1_gpu.minimap2.paf \
$SHASTA_CTG_RACON_I1_GPU > $WORKDIR/Shasta_all_v0.2.0.assembly.racon_i2.includeUnpolished.gpu.fasta"

Job <746108> is submitted to default queue <lowpriority>.


41914.85 sec

### 3.3 round 3

#### 3.3.1 mapping

In [41]:
FLYE_CTG_RACON_I2_GPU=$WORKDIR/Flye_all_trimmed_i1.assembly.racon_i2.includeUnpolished.gpu.fasta
FLYE_CTG_RACON_I2_CPU=$WORKDIR/Flye_all_trimmed_i1.assembly.racon_i2.includeUnpolished.cpu.fasta
SHASTA_CTG_RACON_I2_GPU=$WORKDIR/Shasta_all_v0.2.0.assembly.racon_i2.includeUnpolished.gpu.fasta

In [33]:
bsub -J minimap2 \
-n 20 \
-o $WORKDIR/mapping_minimap2_flye_i2_gpu.out \
-e $WORKDIR/mapping_minimap2_flye_i2_gpu.err \
"minimap2 \
-x map-ont \
-t 20 
$FLYE_CTG_RACON_I2_GPU \
$ONT \
> $WORKDIR/ONT_to_flye_i2_gpu.minimap2.paf"

Job <746443> is submitted to default queue <lowpriority>.


In [34]:
bsub -J minimap2 \
-n 20 \
-o $WORKDIR/mapping_minimap2_flye_i2_cpu.out \
-e $WORKDIR/mapping_minimap2_flye_i2_cpu.err \
"minimap2 \
-x map-ont \
-t 20 \
$FLYE_CTG_RACON_I2_CPU \
$ONT \
> $WORKDIR/ONT_to_flye_i2_cpu.minimap2.paf"

Job <746444> is submitted to default queue <lowpriority>.


In [35]:
bsub -J minimap2 \
-n 20 \
-o $WORKDIR/mapping_minimap2_shasta_i2_gpu.out \
-e $WORKDIR/mapping_minimap2_shasta_i2_gpu.err \
"minimap2 \
-x map-ont \
-t 20 \
$SHASTA_CTG_RACON_I2_GPU \
$ONT \
> $WORKDIR/ONT_to_shasta_i2_gpu.minimap2.paf"

Job <746445> is submitted to default queue <lowpriority>.


#### 3.3.2 racon

In [42]:
bsub -J racon-gpu \
-m wkoppg31 \
-o $WORKDIR/racon_i3_flye_gpu.out \
-e $WORKDIR/racon_i3_flye_gpu.err \
-R gpu \
"/workspace/hraegb/singularity/racon-1.4.7.sif \
--include-unpolished \
-c 2 \
-b \
--cudaaligner-batches 2 \
$ONT \
$WORKDIR/ONT_to_flye_i2_gpu.minimap2.paf \
$FLYE_CTG_RACON_I2_GPU > $WORKDIR/Flye_all_trimmed_i1.assembly.racon_i3.includeUnpolished.gpu.fasta"

Job <747317> is submitted to default queue <lowpriority>.


In [43]:
bsub -J racon-cpu \
-n 40 \
-m wkoppg32 \
-o $WORKDIR/racon_i3_flye_cpu.out \
-e $WORKDIR/racon_i3_flye_cpu.err \
-R gpu \
"/workspace/hraegb/singularity/racon-1.4.7.sif \
--include-unpolished \
-t 40 \
$ONT \
$WORKDIR/ONT_to_flye_i2_cpu.minimap2.paf \
$FLYE_CTG_RACON_I2_CPU > $WORKDIR/Flye_all_trimmed_i1.assembly.racon_i3.includeUnpolished.cpu.fasta"

Job <747318> is submitted to default queue <lowpriority>.


In [44]:
bsub -J racon-gpu \
-o $WORKDIR/racon_i3_shasta_gpu.out \
-e $WORKDIR/racon_i3_shasta_gpu.err \
-m wkoppg34 \
-R gpu \
"/workspace/hraegb/singularity/racon-1.4.7.sif \
--include-unpolished \
-c 2 \
-b \
--cudaaligner-batches 2 \
$ONT \
$WORKDIR/ONT_to_shasta_i2_gpu.minimap2.paf \
$SHASTA_CTG_RACON_I2_GPU > $WORKDIR/Shasta_all_v0.2.0.assembly.racon_i3.includeUnpolished.gpu.fasta"

Job <747319> is submitted to default queue <lowpriority>.
